In [2]:
# coding:utf8
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from tensorflow.python.framework import ops

%matplotlib auto

ops.reset_default_graph() 

#Q+:  name_scope的作用是可以用生成流程图吗？
def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):
    layer_name = 'layer%s'% n_layer
    with tf.name_scope(layer_name): # 意思是这个with里的元素被包括在scope里
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W') # 就是建造一个数组，[行,列]
            tf.summary.histogram(layer_name+'/weights',Weights) # 让weights
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1,name='b')
            tf.summary.histogram(layer_name+'/biases',biases)
        with tf.name_scope('Wx'):
            Wx_plus_b=tf.matmul(inputs,Weights) + biases # 计算
        if activation_function is None: # 如果不指定激励函数
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b) # 如果指定了激励函数
        tf.summary.histogram(layer_name+'/outputs',outputs)
        return outputs

#读取数据
npg= pd.read_csv('data4.csv',header=0)
npg_df = npg.values
npg_arr= np.array(npg_df)
scarler= preprocessing.MinMaxScaler()
npg_arr_minmax=scarler.fit_transform(npg_arr)
x= npg_arr_minmax[:,:2]
y= npg_arr_minmax[:,2:3]

#交叉验证
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
costs = [] 

with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,2],name='x_input')
    ys = tf.placeholder(tf.float32,[None,1],name='y_input')

l1 = add_layer(xs,2,8,n_layer=1,activation_function=tf.nn.relu) # 输入数据是xs,１个输入，8个输出，层数是１，激励函数是relu
predition = add_layer(l1,8,1,n_layer=2,activation_function=None) # 输入数据是l1的输出, 8个输入，１个输出，层数是２，无激励函数

with tf.name_scope('loss'):
    loss = tf.losses.mean_squared_error(ys,predition)
    tf.summary.scalar('loss',loss) # 记录loss的，可以用tensorboard查看

learning_rate=0.2

with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
saver = tf.train.Saver()
 
#初始化权重和阈值
init = tf.global_variables_initializer()
sess = tf.Session()
summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter("npg0/",sess.graph)
sess.run(init)

 
fig = plt.figure() # 用这个画图
ax = fig.add_subplot(1,1,1) # 图片编号
num=np.arange(1,11).reshape(y_train.shape)
ax.scatter(num,y_train) # 点的编号
plt.ion() # 不用停顿

# 训练部分
print("训练开始...")
for epoch in range(5000):
    #为了将loss画图用了一种莫名奇妙的方法
    epoch_cost = 0
    _,cost = sess.run([optimizer,loss],feed_dict={xs:x_train,ys:y_train})
    epoch_cost=cost
    if epoch%100 == 0:
        print("第%3d次训练的损失值：%lf"% (epoch,epoch_cost))
        result = sess.run(summary_op,feed_dict={xs:x_train,ys:y_train})
        writer.add_summary(result,epoch) # 每i步输出一个结果
        try:
            ax.lines.remove(lines[0]) # 去除上次的线
        except Exception:
            pass
        
        predition_value = sess.run(predition,feed_dict={xs:x_train}) # 预测值
        lines = ax.plot(num,predition_value,'r-',lw=2) # 用红色，宽度为５的曲线的形式画出来预测曲线
        plt.pause(0.1)

    if epoch%5 == 0:
        costs.append(epoch_cost)
        
saver.save(sess, 'test-ckpt1/model')

print("训练集均方误差：",sess.run(loss, feed_dict={xs: x_train, ys: y_train}))


# plot the loss
plt.plot(np.squeeze(costs))
plt.ylabel('costs')
plt.xlabel('epochs (per 5)')
plt.title("Learning rate = " + str(learning_rate))
plt.show()

#问题三 可以直接输出测试集的loss吗?
#print ("Test lost:", loss.eval({xs: x_test, ys: y_test}))

# 测试部分
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file("test-ckpt1/model",'weights,biases', None, True)

with tf.name_scope('inputs'):
    xs2 = tf.placeholder(tf.float32,[None,2],name='x_input')
    ys2 = tf.placeholder(tf.float32,[None,1],name='y_input')
with tf.name_scope('parameter'):
    W1 = tf.Variable(tf.random_normal([2,8]),name='W1')
    W2 = tf.Variable(tf.random_normal([8,1]),name='W2')
    b1 = tf.Variable(tf.random_normal([1,8]),name='b1')
    b2 = tf.Variable(tf.random_normal([1,1]),name='b2')

# 把保存的变量提出来赋值给相应的变量
saver = tf.train.Saver({"layer1/weights/W":W1, "layer2/weights/W":W2, "layer1/biases/b":b1, "layer2/biases/b":b2})

a1 = tf.nn.relu(tf.matmul(xs2,W1)+b1)
a2 = tf.matmul(a1,W2)+b2

mse = tf.losses.mean_squared_error(a2,ys2)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, 'test-ckpt1/model')
    #print(sess.run(W1), sess.run(W2))
    print("测试集均方误差：",sess.run(mse, feed_dict={xs2: x_test, ys2: y_test}))
    a2_value=sess.run(a2,feed_dict={xs2: x_test, ys2: y_test})
    #a2_value_train=sess.run(a2,feed_dict={xs2: x_train, ys2: y_train})
    print(a2_value)



Using matplotlib backend: Qt5Agg
训练开始...
第  0次训练的损失值：0.145830
第100次训练的损失值：0.004697
第200次训练的损失值：0.003624
第300次训练的损失值：0.002917
第400次训练的损失值：0.002415
第500次训练的损失值：0.002012
第600次训练的损失值：0.001668
第700次训练的损失值：0.001357
第800次训练的损失值：0.001092
第900次训练的损失值：0.000873
第1000次训练的损失值：0.000683
第1100次训练的损失值：0.000536
第1200次训练的损失值：0.000423
第1300次训练的损失值：0.000339
第1400次训练的损失值：0.000269
第1500次训练的损失值：0.000221
第1600次训练的损失值：0.000185
第1700次训练的损失值：0.000160
第1800次训练的损失值：0.000140
第1900次训练的损失值：0.000126
第2000次训练的损失值：0.000114
第2100次训练的损失值：0.000105
第2200次训练的损失值：0.000096
第2300次训练的损失值：0.000090
第2400次训练的损失值：0.000085
第2500次训练的损失值：0.000080
第2600次训练的损失值：0.000075
第2700次训练的损失值：0.000071
第2800次训练的损失值：0.000066
第2900次训练的损失值：0.000064
第3000次训练的损失值：0.000060
第3100次训练的损失值：0.000057
第3200次训练的损失值：0.000053
第3300次训练的损失值：0.000050
第3400次训练的损失值：0.000047
第3500次训练的损失值：0.000045
第3600次训练的损失值：0.000042
第3700次训练的损失值：0.000040
第3800次训练的损失值：0.000038
第3900次训练的损失值：0.000036
第4000次训练的损失值：0.000035
第4100次训练的损失值：0.000032
第4200次训练的损失值：0.000030
第4300次训练的损失值：0.000029
第

In [4]:
d=scarler.inverse_transform(np.append(x_test,a2_value,axis=1))
print(d,'\n')
e=scarler.inverse_transform(np.append(x_test,y_test,axis=1))
print(e,'\n')
print((e-d)/e,'\n')
print(np.mean(((e-d)/e),axis=0))
%matplotlib auto
plt.plot(d[:,2])
plt.plot(e[:,2])
plt.xticks(np.arange(1,9,1))

[[   0.3           3.3         160.42510212]
 [   0.3           3.3         160.42510212]
 [   0.3           4.1         169.08303884]
 [   0.35          4.9         186.07722514]
 [   0.4           4.49        278.13069773]
 [   0.3           2.5         134.1075995 ]
 [   0.33          3.3         194.7580187 ]
 [   0.4           5.3         242.00791965]] 

[[   0.3     3.3   161.2 ]
 [   0.3     3.3   161.2 ]
 [   0.3     4.1   168.6 ]
 [   0.35    4.9   206.3 ]
 [   0.4     4.49  284.1 ]
 [   0.3     2.5   163.9 ]
 [   0.33    3.3   174.4 ]
 [   0.4     5.3   242.6 ]] 

[[ 0.          0.          0.00480706]
 [ 0.          0.          0.00480706]
 [ 0.          0.         -0.002865  ]
 [ 0.          0.          0.09802605]
 [ 0.          0.          0.02101127]
 [ 0.          0.          0.18177182]
 [ 0.          0.         -0.11673176]
 [ 0.          0.          0.00244056]] 

[ 0.          0.          0.02415838]
Using matplotlib backend: Qt5Agg


([<matplotlib.axis.XTick at 0x190188b4160>,
 <a list of 8 Text xticklabel objects>)